In [4]:
import pandas as pd

df = pd.read_csv("datasets/fake_or_real_news.csv")

df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import re

nltk.download("punkt")
nltk.download("stopwords")

stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

def preprocess_text(text):
    # Küçük harfe dönüştür
    text = text.lower()
    # Noktalama işaretlerini kaldır
    text = re.sub(r'[^\w\s]', '', text)
    # Tokenization
    tokens = word_tokenize(text)
    # Stop words'leri kaldır ve stemming uygula
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

df['processed_text'] = df['text'].apply(preprocess_text)

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1018)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1018)>


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/omerasafkarasu/nltk_data'
    - '/Users/omerasafkarasu/Desktop/fake-news-checker/.venv/nltk_data'
    - '/Users/omerasafkarasu/Desktop/fake-news-checker/.venv/share/nltk_data'
    - '/Users/omerasafkarasu/Desktop/fake-news-checker/.venv/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [93]:
df.head()

,Unnamed: 0,title,text,label,processed_text
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE,daniel greenfield shillman journal fellow free...
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE,googl pinterest digg linkedin reddit stumbleup...
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL,us secretari state john f kerri said monday st...
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE,kayde king kaydeek novemb 9 2016 lesson tonigh...
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL,primari day new york frontrunn hillari clinton...


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features = 5000)
x = tfidf_vectorizer.fit_transform(df["processed_text"]).toarray()
y = df["label"].values

NameError: name 'df' is not defined

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB

# Veriyi eğitim ve test setlerine ayır
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Logistic Regression modeli

model = LogisticRegression()
#model = GaussianNB()
model.fit(x_train, y_train)

# Tahmin yap
y_pred = model.predict(x_test)
print("Tahmin: ", y_pred)

# Doğruluk değerlendirmesi
accuracy = accuracy_score(y_test, y_pred)
print(f'Model Doğruluğu: {accuracy * 100:.2f}%')

NameError: name 'x' is not defined

In [121]:
import joblib

# Modeli kaydet
joblib.dump(model, 'fake_news_model.pkl')

# Modeli yükle
model = joblib.load('fake_news_model.pkl')

def ask(text):
    processed_text = preprocess_text(text)
    vectorized_text = tfidf_vectorizer.transform([processed_text]).toarray()
    prediction = model.predict(vectorized_text)
    print(prediction)
    print('Gerçek Haber' if prediction[0] == "REAL" else 'Sahte Haber')

In [158]:
import random
num = random.randint(0, 100)

test = df["text"][num]
print(df["label"][num])
ask(test)

FAKE
['FAKE']
Sahte Haber


In [133]:
ask("Harry potter is dead")

['FAKE']
Sahte Haber


In [135]:
ask("Harry Potter is live")

['FAKE']
Sahte Haber


In [136]:
ask("Biden is live")

['FAKE']
Sahte Haber


In [137]:
ask("Trump is a president of America")

['FAKE']
Sahte Haber


In [143]:
ask("""Sir Keir called Zelensky on Wednesday evening and told him it was "perfectly reasonable" for Ukraine to "suspend elections during wartime as the UK did during World War Two", Downing Street said. The US president had earlier criticised Zelensky, saying he had done a "terrible job" and claiming "he refuses to have elections" in Ukraine. Zelensky's five-year term was due to end in May 2024, but elections have been suspended since martial law was declared after Russia's invasion.""")

['FAKE']
Sahte Haber


In [161]:
ask(" wilamid")

['FAKE']
Sahte Haber
